In [1]:
import matplotlib.pyplot as plt
from numpy import sin, cos, sqrt, pi, linspace, arange, deg2rad, rad2deg, array, arcsin, arccos,sort, argsort, argwhere, argmin, argmax, interp, concatenate
from scipy.spatial import distance
import warnings
warnings.simplefilter('ignore')

def sec(x):
    return 1/cos(x)
def tan(x):
    return sin(x)/cos(x)

In [32]:
case  = 'wat'


if case=='atm':
    theta  = 0
    n      = 1.00029
    c      = 299792458/n
    h      = 25000
    L      = 100
    v      = n * c
    times  = linspace(1e-6,1e-4,100000,endpoint=True)
    
elif case=='wat':
    n      = 1.33
    c      = 299792458/n
    h      = 4.5
    D      = 7.3   #--------------------- diameter of tank
    R      = 7.3/2 #--------------------- radius of tank
    v      = n * c
    times  = linspace(1e-10,1e-7,100000)
    
    c1     = (0, 0, 0)  #------------------------------------ Central    PMT number 1 
    c2     = (1.85*cos(2*pi/3) , 1.85*sin(2*pi/3), 0) #------ Non-Radial PMT number 2
    c3     = (1.85*cos(4*pi/3) , 1.85*sin(4*pi/3), 0) #------ Non-Radial PMT number 3
    c4     = (1.85*cos(0     ) , 1.85*sin(0     ), 0) #------ Radial     PMT number 4
    x1,y1  = (1.5,1.5) #----------------------------------------- Coordinates of particle at top tanker lid      #------PARAM
    theta  = 180   #----------------------------------------- Zenith  varies from 90 to 180                    #------PARAM
    phi    = 0    #----------------------------------------- Azimuth varies from  0 to 360                    #------PARAM
    #plotme = 'phi vs t' #------PARAM
    theta  = deg2rad(theta) #-------------------------------- Convert zenith to radians
    phi    = deg2rad(phi  ) #-------------------------------- Convert azimuth to radians

    x2     = x1 + h * tan(theta) * cos(phi) # ------------------- Calculate bottom x coordinate
    y2     = y1 + h * tan(theta) * sin(phi) #-------------------- Calculate bottom y coordinate
    cp     = (x2,y2,0)
    print('A = {}'.format((x1,y1,h)))
    print('B = {}'.format(cp))

    d1     = distance.euclidean(cp,c1)  #------------------------ Calculate the distances of the point 
    d2     = distance.euclidean(cp,c2)  #------------------------ where the particle hits the bottom of 
    d3     = distance.euclidean(cp,c3)  #------------------------ the tank from the 4 PMTs
    d4     = distance.euclidean(cp,c4)
    theta2    = pi - theta
    del theta
    theta     = theta2
    print('Particle distances from PMTs    : {} m,   {} m,   {} m,   {} m'.format( round(d1,3),round(d2,3),round(d3,3),round(d4,3)))

A = (1.5, 1.5, 4.5)
B = (1.4999999999999996, 1.5, 0)
Particle distances from PMTs    : 2.121 m,   2.427 m,   3.938 m,   1.54 m


In [35]:
def entry_brightness(h,c,v,theta=0,L=2.121):
    z        = h/cos(theta)
    #print("z:",z)
    k        = (sqrt( L**2 + z**2 - 2*z*L*cos(pi/2-theta)))
    t_shower = 0
    t_light  = k/c
    t_total  = t_shower + t_light
    tt       = t_total
    num1_z   = -c**2*tt*v + c**2*h*sec(theta) - L*v**2*sin(theta)
    num2_z   = sqrt(v**2* ( c*c*L*L - L*L*v*v + c*c*tt*tt*v*v - 2*c*c*h*tt*v*sec(theta) + c*c*h*h*sec(theta)**2 + 2*c*c*L*tt*v*sin(theta) + L*L*v*v*sin(theta)**2 - 2*c*c*h*L*tan(theta)))
    zp       = ((num1_z - num2_z) / den)
    zm       = ((num1_z + num2_z) / den)
    v_A      = v*v*(tt*v-h*sec(theta)+L*sin(theta))
    v_B      = num2_z
    vp       = (c*c*v) * (1+(v_A/v_B)) / (-den)
    vm       = (c*c*v) * (-1+(v_A/v_B)) / den        
    kp       = sqrt( L**2 + zp**2 - 2*zp*L*cos(pi/2-theta))
    km       = sqrt( L**2 + zm**2 - 2*zm*L*cos(pi/2-theta))
    alphap   = (arccos( (zp*zp + kp*kp - L*L) / (2*zp*kp) ))
    alpham   = (arccos( (zm*zm + km*km - L*L) / (2*zm*km) ))
    vtp      = vp * sin(alphap)
    vtm      = vm * sin(alpham)
    omegap   = (vtp / kp)
    omegam   = (vtm / km)
    bp       = abs(omegap / (kp**2))
    bm       = abs(omegam / (km**2))
    #if (zm*cos(theta)<=0):
    #    pass
    return bm

entry_brightness(h,c,v,theta=0,L=2.121)

48677747.1866397

In [36]:
%matplotlib
plt.figure(figsize=(12,9))

plotme = 'b vs phi'
if 'b' in plotme:
    ptype = str(input("Do you want normalized brightness? Enter Y/y or N/n"))
    
distances = [d1,d2]
colors    = ['r','k']
detectors = [1,2]

for ds, color, detector in zip(distances, colors, detectors):

    T, ZP, ZM, BP, BM, PHIP, PHIM = [], [], [], [], [], [], []
    L      = ds
    num_zc = -sqrt(-c**4 * L**2 * cos(theta)**2 + c**2 * L**2 * v**2 * cos(theta)**2) + c**2*L*sin(theta) - L*v**2*sin(theta)
    den    = (c**2-v**2)
    zc     = (num_zc/den)
    zc_act = zc * cos(theta)
    
    print('First RID Height: {} m'.format(zc_act))
    entry_bright = entry_brightness(h,c,v,theta,L=0)

    for iii,t in enumerate(times):

        z        = h/cos(theta) - v*t
        k        = (sqrt( L**2 + z**2 - 2*z*L*cos(pi/2-theta)))
        t_shower = t
        t_light  = k/c
        t_total  = t_shower + t_light
        tt       = t_total

        num1_z   = -c**2*tt*v + c**2*h*sec(theta) - L*v**2*sin(theta)
        num2_z   = sqrt(v**2* ( c*c*L*L - L*L*v*v + c*c*tt*tt*v*v - 2*c*c*h*tt*v*sec(theta) + c*c*h*h*sec(theta)**2 + 2*c*c*L*tt*v*sin(theta) + L*L*v*v*sin(theta)**2 - 2*c*c*h*L*tan(theta)))

        zp       = ((num1_z - num2_z) / den)
        zm       = ((num1_z + num2_z) / den)

        v_A      = v*v*(tt*v-h*sec(theta)+L*sin(theta))
        v_B      = num2_z

        vp       = (c*c*v) * (1+(v_A/v_B)) / (-den)
        vm       = (c*c*v) * (-1+(v_A/v_B)) / den        

        kp       = sqrt( L**2 + zp**2 - 2*zp*L*cos(pi/2-theta))
        km       = sqrt( L**2 + zm**2 - 2*zm*L*cos(pi/2-theta))

        alphap   = (arccos( (zp*zp + kp*kp - L*L) / (2*zp*kp) ))
        alpham   = (arccos( (zm*zm + km*km - L*L) / (2*zm*km) ))

        phip     = 90+theta-rad2deg(alphap)
        phim     = 90+theta-rad2deg(alpham)            
        
        #print(  (zp*zp + kp*kp - L*L) / (2*zp*kp) ,  (zm*zm + km*km - L*L) / (2*zm*km) )
        
        vtp      = vp * sin(alphap)
        vtm      = vm * sin(alpham)

        omegap   = (vtp / kp)
        omegam   = (vtm / km)

        bp       = abs(omegap / (kp**2))
        bm       = abs(omegam / (km**2))

        ZM.append(zm*cos(theta)) # TAKE CARE OF THIS
        ZP.append(zp*cos(theta)) # TAKE CARE OF THIS
        BM.append(bm)            
        BP.append(bp)
        PHIP.append(phip)
        PHIM.append(phim)
        T.append(tt)

    ZP, ZM, T, BP, BM, PHIP, PHIM = array(ZP), array(ZM), array(T), array(BP), array(BM), array(PHIP), array(PHIM)
    print("ZP: ",ZP)
    print("ZM: ",ZM)
    #print("BP[-1]: ",BP[-1])
    Z1n = ZP
    Z2n = ZM
    Tn  = T * 1e+9
    TT  = (Tn-Tn.min())
    
    if 'b' in plotme:
        if ptype==('y' or 'Y'):
            print('Entry Brightness:', entry_bright)
            BM = BM/entry_bright
            BP = BP/entry_bright
        else:
            pass
       
    #if detector==2:
    #cond1 = (ZP<=h*cos(theta))
    #ZM, ZP, TT, BM, BP, PHIM, PHIP = ZM[cond1], ZP[cond1], TT[cond1], BM[cond1], BP[cond1], PHIM[cond1], PHIP[cond1]
    #print(TT.shape, PHIM.shape, PHIP.shape)
    #cond2 = (ZM>=0)
    #ZM, ZP, TT, BM, BP, PHIM, PHIP = ZM[cond2], ZP[cond2], TT[cond2], BM[cond2], BP[cond2], PHIM[cond2], PHIP[cond2]
    #print(TT.shape, PHIM.shape, PHIP.shape)
    #if BP[-1]:
    #    print("BP[-1]: ",BP[-1]) 
    
    if detector==1:
        mylabel=None#'Central Detector'
    else:
        mylabel='Surrounding Detectors'
            
    if plotme == 'x vs t':
        plt.plot(TT[Z1n<=h*cos(theta)], Z1n[Z1n<=h*cos(theta)], c=color, ls='--')
        plt.plot(TT[Z1n<=h*cos(theta)], Z2n[Z1n<=h*cos(theta)], c=color, ls='-' , label=r'Detector {}'.format(detector))
        plt.axhline(zc_act, c='k', ls=':')
        plt.ylim(0,h)
        if L>1e-10:
            plt.text(4, round(zc_act,2)+0.075, r'$z_c = {} \; m$'.format(round(zc_act,2)), fontsize=16)
        plt.xlabel(r'time $( t_{total} - t_{min} )$ (ns)', fontsize=18)
        plt.ylabel(r'image height $z_{\pm}$ (m)'         , fontsize=18).set_rotation(90)
        plt.tick_params(axis='both', direction='in', labelsize=18)
    
    elif plotme == 'b vs t':
        plt.plot(TT[condition], BM[condition], c=color, ls='--')
        plt.plot(TT[condition], BP[condition], c=color, ls='-' , label=r'Detector {}'.format(detector))
        plt.axhline(1, c='k', ls=':')
        plt.xlabel(r'time $( t_{total} - t_{min} )$ (ns)', fontsize=18)
        plt.ylabel(r'brightness ($b/b_{ground}$) '       , fontsize=18).set_rotation(90)
        plt.tick_params(axis='both', direction='in', labelsize=18)        
        plt.yscale('log')

    elif plotme == 'b vs phi':
        plt.plot(PHIM, BM, c=color, ls='-.')
        plt.plot(PHIP, BP, c=color, ls='-' , label=mylabel)
        plt.xlabel(r'angular locations $\phi_{pm}\;(in\;degrees)$', fontsize=18)
        plt.ylabel(r'brightness ($b/b_{entry}$) '       , fontsize=18).set_rotation(90)
        plt.tick_params(axis='both', direction='in', labelsize=18)        
        plt.yscale('log')
        plt.xlim(0,90)

    elif plotme == 'phi vs t':
        plt.plot(TT, PHIM, c=color, ls='-.')
        plt.plot(TT, PHIP, c=color, ls='-' , label=mylabel)
        plt.xlabel(r'time $( t_{total} - t_{min} )$ (ns)', fontsize=18)
        plt.ylabel(r'angular locations $\phi_{pm}\;(in\;degrees)$'       , fontsize=18).set_rotation(90)
        plt.tick_params(axis='both', direction='in', labelsize=18)
        plt.ylim(0,90.01)
        plt.xlim(0,1.2)
    break
    
plt.legend()
plt.show()

Using matplotlib backend: Qt5Agg


Do you want normalized brightness? Enter Y/y or N/n y


First RID Height: 2.419197507879486 m


No handles with labels found to put in legend.


ZP:  [  4.47002075   4.46972126   4.46942176 ... 180.19966519 180.20177624
 180.20388728]
ZM:  [  1.0199044    1.02004688   1.02018937 ... -25.47864681 -25.4789463
 -25.4792458 ]
Entry Brightness: 6.086449678926514e-08


In [22]:
BP[-1]

26227790.457191385

In [23]:
ZP

array([  4.47002075,   4.46972126,   4.46942176, ..., 180.12673594,
       180.12884784, 180.13095974])

In [38]:
plt.plot(ZP)
plt.plot(ZM)
plt.ylim(0,5)

(0, 5)